In [3]:
#Importing essentials libraries

import csv
import numpy
import matplotlib.pyplot as plt
import pandas
import math

from sklearn.preprocessing import MinMaxScaler, StandardScaler, Normalizer, RobustScaler
from sklearn.metrics import mean_squared_error

# forecasting package
from fbprophet import Prophet

import collections
import datetime
import scipy
from scipy import stats

import plotly
import plotly.plotly as py
from plotly.graph_objs import Scatter, Heatmap, Layout
plotly.offline.init_notebook_mode(connected=True)

In [4]:
#Function for building the model

# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
  dataX, dataY = [], []
  for i in range(len(dataset)-look_back-1):
    a = dataset[i:(i+look_back), 0]
    dataX.append(a)
    dataY.append(dataset[i + look_back, 0])
  return numpy.array(dataX), numpy.array(dataY)


def remove_extreme_vals(keys,values):
    quartiles = numpy.percentile(values, [5,95])
    minVal = quartiles[0] #5% of the values are inferior or equal to his values.
    maxVal = quartiles[1] #95% of the values are inferior or equal to this value
    jours_x = []
    jours_y = []
    
    print(minVal)
    print(maxVal)
    
    for i in range(0, len(values) ):
        if minVal < values[i] < maxVal:
            jours_x.append(keys[i])
            jours_y.append(values[i])
            
    return jours_x,jours_y

def dividers(x):
    liste_dividers = []
    
    for i in range(1,x):
        if ( x%i == 0 ):
            liste_dividers.append(i)
            
    return liste_dividers

In [5]:
#Read dataset file
file = pandas.read_csv("../data/improved_spvm_2015-2019.csv")

In [6]:
#Creating the dataset to train and test.
jours = file["JOUR"]
jours = list(jours)

#Sort the list to be able to see something.
jours = sorted(jours)

jours = collections.Counter(jours)

jours_x = list(jours.keys())
jours_y = list(jours.values())

#We make sure nothing has been dropped.
assert len(jours_x) == len(jours_y)

dataframe = pandas.DataFrame({ 'ds' : jours_x, 'y': jours_y })

df_train = dataframe[0:int(0.70*dataframe.shape[0])]
df_test = dataframe[int(0.70*dataframe.shape[0])+1:]

#We make sure again nothing has been dropped.
assert (df_train.shape[0] + df_test.shape[0])

print("Preparing dataset finished !")

Preparing dataset finished !


In [7]:
# initialize the Prophet model object
m = Prophet()

# fit the model
m.fit(df_train)


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
/Users/moumine/anaconda3/lib/python3.7/site-packages/pystan/misc.py:399: FutureWarning:

Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.



In [8]:
#Predictions

future = m.make_future_dataframe(periods= df_test.shape[0] )
forecast = m.predict(future)

#fig1 = m.plot(forecast)
#fig2 = m.plot_components(forecast)

print(forecast.head())

          ds      trend  yhat_lower  yhat_upper  trend_lower  trend_upper  \
0 2015-01-01  98.819788   67.564381  101.027531    98.819788    98.819788   
1 2015-01-02  98.810021   65.695761   99.128846    98.810021    98.810021   
2 2015-01-03  98.800255   54.950846   87.888203    98.800255    98.800255   
3 2015-01-04  98.790488   56.499865   89.158854    98.790488    98.790488   
4 2015-01-05  98.780721   76.868259  109.797906    98.780721    98.780721   

   additive_terms  additive_terms_lower  additive_terms_upper     weekly  \
0      -14.250254            -14.250254            -14.250254   1.656829   
1      -15.921367            -15.921367            -15.921367   0.175019   
2      -26.858390            -26.858390            -26.858390 -10.754717   
3      -26.013718            -26.013718            -26.013718 -10.075110   
4       -5.340579             -5.340579             -5.340579  10.273695   

   weekly_lower  weekly_upper     yearly  yearly_lower  yearly_upper  \
0      1

In [15]:
#Showing predictions

# Create traces
trace_pred = Scatter(
    x = df_test['ds'],
    y = numpy.array(forecast['yhat'], dtype='uint8'),
    mode = 'markers',
    name = 'Predictions'
)

trace_normal = Scatter(
    x = df_test['ds'],
    y = df_test['y'],
    mode = 'markers',
    name = 'Actual Values'
)

data = [trace_pred, trace_normal]

plotly.offline.iplot(data, filename='Res')



In [ ]:
equals = numpy.intersect1d(df_test['y'], normal_vals)
print(len(equals))
print(len(predictions))
print(len(normal_vals))
print(max(predictions))
print(max(normal_vals))